## Hertz Contact
#### Contact between a sphere and a half-space

An elastic sphere of radius $R$ indents an elastic half-space where total deformation is $d$, causing a contact area of radius[1]

![Alt Text](/Users/lizichen/Master-internship-repo-at-Institut-dAlembert/Week2/Contact_sphere-plane.jpg)

$$
a=\sqrt{R d}
$$


The maximum shear stress occurs in the interior at $z \approx 0.49 a$ for $\nu=0.33$


We want to apply this solution for a non-friction Contact between a sphere and a half-space and 

In [41]:
import numpy as np
import matplotlib as plt
import time

The applied force $F$ is related to the displacement $d$ by

$$
F=\frac{4}{3} E^* R^{\frac{1}{2}} d^{\frac{3}{2}}
$$

where 

$$
\frac{1}{E^*}=\frac{1-\nu_1^2}{E_1}+\frac{1-\nu_2^2}{E_2}
$$

and $E_1, E_2$ are the elastic moduli and $\nu_1, \nu_2$ the Poisson's ratios associated with each body.


The radius of the circle is related to the applied load $F$ by the equation

$$
a^3=\frac{3 F R}{4 E^*}
$$

In [42]:
# We define the force as F, we give F as input
F = np.linspace(0, 1e6, 1000)


# We define the radius of the elastic sphere as R
R = 1
# We define the elastic modulus of the sphere as E1, and the elastic modulus of the surface as E2
E1 = 1e5
E2 = 1e3
# We define the Poisson ratio of the sphere as v1, and the Poisson ratio of the surface as v2
v1 = 0.33
v2 = 0.33
# We define the reduced elastic modulus as E_star
E_star = 1 / ((1 - v1**2) / E1 + (1 - v2**2) / E2)
# We define the contact radius as a
a = 3*F*R/(4*E_star)

# We define the domain is L^2
L = 2


## Q1: We apply F correctly?

Here we apply F as an array 

The total deformation $d$ is related to the maximum contact pressure by

$$
d=\left(\frac{9 F^2}{16 E^{* 2} R}\right)^{\frac{1}{3}}
$$

In [43]:
d_analytical = a**2 / R

#d_analytical

#d_analytical2 = (9 * F**2 / (16 * E_star**2 * R))**(1/3)

#d_analytical2

## Q1: Here actually  we just have the total deformation $d$, how can we get the analytical deformation distribution

Here we mesh the domain, and give the coordinate of the center of contact zone as (0,0),then $r$ can be given as $ \sqrt{\left(x-x_0\right)^2+\left(y-y_0\right)^2} $.

The distribution of normal pressure in the contact area as a function of distance from the center of the circle is 

$$
p(r)=p_0\left(1-\frac{r^2}{a^2}\right)^{\frac{1}{2}}
$$

where $p_0$ is the maximum contact pressure given by

$$
p_0=\frac{3 F}{2 \pi a^2}=\frac{1}{\pi}\left(\frac{6 F E^{* 2}}{R^2}\right)^{\frac{1}{3}}
$$


In [44]:
#Here we define p0 as the reference pressure
p0 = (1/np.pi) * (6*F*(E_star**2)/(R**2))**(1/3) 

'''
#We generate a 2D coordinate space
n = 100
m = 100

x, y = np.meshgrid(np.linspace(0, L, n, endpoint=False), np.linspace(0, L, m, endpoint=False))#notice here that we use endpoint=False to avoid having the last point

#Here we define the pressure distribution
def pressure_distribution(x, y, x0, y0, p0):
    r = np.sqrt((x-x0)**2 + (y-y0)**2)
    p = p0 * (1 - (r/a)**2)**0.5
    p[r > a] = 0
'''
def pressure_distribution(x, y, x0, y0, a, p0):
    r2 = (x-x0)**2 + (y-y0)**2
    if r2 <= a**2:        
        return p0*np.sqrt(1-r2/a**2)
    else:
        return 0
pressure_distribution = np.vectorize(pressure_distribution)#try to vectorize the function
# Any benefit of vectorizing the function?



x0 = 0
y0 = 0

#We calculate test pressure on the grid
test_pressure = pressure_distribution(x, x0, p0)
#We generate the pressure distribution in Fourier space
pressure_fourier = np.fft.fft2(test_pressure, norm='ortho')
# Visualization of the Fourier-transformed pressure
fig, ax = plt.subplots()
extent = [x.min(), x.max(), y.min(), y.max()]
im = ax.imshow(np.log(np.abs(pressure_fourier)), extent=extent, origin='lower')#not so clear with abs here # That means we take the norm of the complex number
plt.colorbar(im, ax=ax)
plt.title('Log of Absolute Value of Fourier-transformed Pressure')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


'''#Try this figure
pressure_img = plt.imshow(P/p0, extent=[x_min + dx/2, x_max, y_min + dx/2, y_max], 
                            origin='lower', interpolation='bicubic', cmap=plt.cm.inferno, vmax = 1)
fig.colorbar(pressure_img, ax=ax[0], label='Pressure', orientation="horizontal")
plt.set_title('Pressure Distribution')
plt.set_xlabel('x')
plt.set_ylabel('y')
'''

TypeError: pressure_distribution() missing 3 required positional arguments: 'y0', 'a', and 'p0'

## Note, $p(r>a)$ = 0 

We give the displacement by convolution integral:

$$
u(x, y)=\iint \frac{p\left(x^{\prime}, y^{\prime}\right)}{\sqrt{\left(x-x^{\prime}\right)+\left(y-y^{\prime}\right)^2}} \cdot \frac{1}{\pi E^*} d x^{\prime} d y^{\prime}
$$

We take advantage of the property of convolution product:

$$
\mathcal{F}(u)=\mathcal{F}(f) \cdot \mathcal{F}(p)
$$

We define our Kernel function in fourrier domain:

$$
\mathcal{F}(f)=\frac{2}{E^* \sqrt{q_x^2+q_y^2}}
$$

In [ ]:
#we define the frequency with q_x and q_y
q_x = 2 * np.pi * np.fft.fftfreq(n, d=L/n)
q_y = 2 * np.pi * np.fft.fftfreq(m, d=L/m)
QX, QY = np.meshgrid(q_x, q_y)

kernel_fourier = np.zeros_like(QX)  # Initialize the kernel array
#non_zero_indices = (QX**2 + QY**2) != 0  # Avoid division by zero
kernel_fourier = 2 / (E_star * np.sqrt(QX**2 + QY**2))
kernel_fourier[0, 0] = 0  # Set the zero frequency component to zero

We define our 

In [ ]:
#Calculate the displacement field in fourrier space
displacement_fourrier = pressure_fourier * kernel_fourier

#We calculate the displacement field in real space
displacement_real = np.fft.ifft2(displacement_fourrier, norm='ortho')


# Since displacement_analytical is a large array, we won't print it directly to avoid clutter.
# Instead, let's visualize the real part of the displacement in real space for comparison.
plt.figure()
plt.imshow(np.real(displacement_real), extent=extent, origin='lower')
plt.colorbar()
plt.title('Real Part of Displacement Field in Real Space')
plt.xlabel('x')
plt.ylabel('y')
plt.show()


We want to compute the displacement for different loading correponds to small contact zone and large contact zone.

##### Reference:

[1] https://en.wikipedia.org/wiki/Contact_mechanics